In [1]:
import sys
sys.path.append("/data/powermeter_dlt645/meter-dlt645")

from test_dlt645 import *
import test_dlt645

In [2]:
test_dlt645._main(None)

Run timestamp is 2020-10-24 00:37:22

---> No port specified. Use default /dev/ttyUSB0.

---> No meter address, will use broadcast mode to retrieve.

--- Read meter address with broadcase mode ---
Meter address: 000102866265

--- Read date ---
Date: 20-10-24 Sat

--- Read time ---
Time: 00:27:07

--- Read voltage ---
Voltage: 239.5 V

--- Read current ---
Current: 0.816 A

--- Read power ---
Power: 0.1878 kW

--- Read battery voltage ---
Battery voltage: 3.65 V

--- Read last outage timestamp of N = 1 ---
Last outage timestamp: 20-10-13 16:18:16 ----> 20-10-13 16:27:34 

--- Read energy (Current Month) ---
Total: 19826.25 kWh

--- Read energy (Current Month) ---
Peak: 12031.59 kWh

--- Read energy (Current Month) ---
Trough: 7794.66 kWh

--- Read energy (Last 1 Month) ---
Total: 19483.45 kWh

--- Read energy (Last 1 Month) ---
Peak: 11785.79 kWh

--- Read energy (Last 1 Month) ---
Trough: 7697.65 kWh


